In [ ]:
# !pip install -q diffusers transformers accelerate torch pandas pillow

In [ ]:
import os
import pandas as pd
import torch
from diffusers import StableDiffusionXLPipeline
from PIL import Image

# model_id = "segmind/SSD-1B"
model_id = "stabilityai/stable-diffusion-xl-base-1.0"
# model_id = "stabilityai/sdxl-turbo"
# model_id = "stabilityai/stable-diffusion-1b"

device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.float16 if device == "cuda" else torch.float32

os.makedirs("generated", exist_ok=True)

Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.


In [ ]:
pipe = StableDiffusionXLPipeline.from_pretrained(
    model_id,
    torch_dtype=dtype
).to(device)

pipe.enable_attention_slicing()

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!


In [ ]:
df = pd.read_csv("pages.csv").head(100)

categories = set()
for c in df["categories"].dropna():
  for cat in str(c).split(","):
    categories.add(cat.strip())

print(f"Unique Categories: {len(categories)}")

Unique Categories: 4


In [ ]:
def image_prompt(category):
    return (
        f"Professional high-quality visual representation of {category}, "
        f"modern branding style, clean composition, realistic lighting, "
        f"commercial design, premium look, ultra-sharp, minimal clutter"
    )

def banner_prompt(category):
    return (
        f"Professional high-quality visual representation of {category}, "
        f"modern branding style, clean composition, realistic lighting, "
        f"commercial design, premium look, ultra-sharp, minimal clutter"
    )

negative_prompt = """
photorealistic, photography, 3D render, clay style, sketch, watercolor, oil painting,
cartoonish, childish, messy composition, cluttered background, excessive details,
text, letters, logos, branding, watermark, signature,
blurry, low resolution, noisy, grainy, distorted shapes, bad proportions,
overly dramatic lighting, dark mood, saturated colors
"""

In [ ]:
for category in categories:
  safe_name = category.replace("/", "_").replace("&", "and")

  print(f"Generating for: {category}")
  img = pipe(
      prompt = image_prompt(category),
      negative_prompt = negative_prompt,
      height = 1024,
      width = 1024,
      num_inference_steps = 15,
      guidance_scale = 7.5
  ).images[0]

  img.save(f"generated/{safe_name}_image.png")

  banner = pipe(
      prompt = banner_prompt(category),
      negative_prompt = negative_prompt,
      height = 400,
      width = 1584,
      num_inference_steps = 15,
      guidance_scale = 7.5
  ).images[0]

  banner.save(f"generated/{safe_name}_banner.png")

print("Done.")

Generating for: Insurance Agents


  0%|          | 0/15 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/diffusers/pipelines/stable_diffusion_xl/pipeline_stable_diffusion_xl.py:748: FutureWarning: `upcast_vae` is deprecated and will be removed in version 1.0.0. `upcast_vae` is deprecated. Please use `pipe.vae.to(torch.float32)`. For more details, please refer to: https://github.com/huggingface/diffusers/pull/12619#issue-3606633695.
  deprecate(


  0%|          | 0/15 [00:00<?, ?it/s]

Generating for: Car Repairs And Services


...

  0%|          | 0/15 [00:00<?, ?it/s]

Generating for: Car Accessories


  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

Done.


In [ ]:
!zip -r folder.zip generated
from google.colab import files
files.download('folder.zip')

  adding: generated/ (stored 0%)
  adding: generated/Car Repairs And Services_image.png (deflated 0%)
  adding: generated/Insurance Agents_banner.png (deflated 0%)
  adding: generated/Insurance Agents_image.png (deflated 0%)
  adding: generated/Budget Hotels_banner.png (deflated 0%)
  adding: generated/Car Accessories_image.png (deflated 0%)
  adding: generated/Car Accessories_banner.png (deflated 0%)
  adding: generated/Budget Hotels_image.png (deflated 0%)
  adding: generated/Car Repairs And Services_banner.png (deflated 0%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>